# This notebook does grid search of fits and gives RMSE fitting values of each

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.integrate as integrate
from scipy.optimize import curve_fit
plt.style.use('coolplot.mplstyle')

df = pd.read_csv('muon_counter_data.csv')
df['average_cpm_coincidental'] = df['average_cpm_coincidental'] - 0 ### omit the correction, because its taken in a noisy cave
######################################################

########################### Sub-Functions #######################

def fudger(height, width, length, fudge_distance):
    
    height +=  2*fudge_distance
    width -= 2*fudge_distance
    length -= 2*fudge_distance
    
    phi = 2*np.arctan(width/height) # gives the aperture angles of the detector in radians    
    alpha = 2*np.arctan(length/height)  
    
    # print ('The angle of the width aperture is:',phi*180/np.pi,"degrees")                                                    
    # print ('The angle of the length aperture is:',alpha*180/np.pi,"degrees")
    
    return height, width, length, phi, alpha


######################################################

def cos_func_squared (y,x,zenith_angle):
    
    a = 10 # in km  atmosphere height
    r = 6400 # in km  radius of the planet earth
    
    new_cos = abs(np.cos(np.arcsin(np.sin(zenith_angle + x ) * np.cos(y/2))))
    # output = (new_cos**2)   ###test
    output = (a/(-r*new_cos + np.sqrt(a**2 + 2*a*r + (r*new_cos)**2)))**2
    
    return output


######################################################

def integral_of_the_flux (zenith_angle, alpha, phi, height, width, length, F0):
    """Here we define the integrand and we integrate to get the flux through the detectors for a given zenith angle."""
    
    alim = -alpha/2
    blim = alpha/2
    
    alim_2 = -phi/2
    blim_2 = phi/2
        
    f = lambda y,x: np.sin(x + np.pi/2) * np.cos(y/2) * (width - (height*abs(np.tan(y)))) * (length - (height*abs(np.tan(x)))) * cos_func_squared (y,x,zenith_angle)
    
    result = F0 * integrate.dblquad(f, alim, blim, alim_2, blim_2)[0]
    
    return result


########################################################## functions ###############····························


def generator_func_arr (argument_array, F0, fudge_distance, height, width, length):
    """ This hunction geneerates prediction values. For Arrays!
    INPUT: F0 - the intensity of the flux
        fudge distance - the amount of dead zone arounf the detector
        height, width, length - geometrical dimemsions of the detector
        argument_array - an array with the feature (angles in this case)
    OUTPUT: an array of the predicted values"""
    
    height, width, length, phi, alpha = fudger(height, width, length, fudge_distance) # transforms the geometrical parameters and gives derived geometrical values
    
    integral_table = np.zeros(len(argument_array))  # table to store the computed vales
    argument_array=(np.pi/180)*argument_array # the argument values in radians
    
    for i in range(len(argument_array)):
        integral_table[i] = integral_of_the_flux (argument_array[i], alpha, phi, height, width, length, F0) # compute the value, based on the given parameters
    
    return integral_table


###########################

# def generator_func_single (argument, F0, fudge_distance, height, width, length):
#     """ This hunction geneerates prediction values. For Arrays!
#     INPUT: F0 - the intensity of the flux
#         fudge distance - the amount of dead zone arounf the detector
#         height, width, length - geometrical dimemsions of the detector
#         argument_array - an array with the feature (angles in this case)
#     OUTPUT: an array of the predicted values"""
    
#     height, width, length, phi, alpha = fudger(height, width, length, fudge_distance) # transforms the geometrical parameters and gives derived geometrical values
    
#     argument=(np.pi/180)*argument # the argument values in radians
    
#     return integral_of_the_flux (argument, alpha, phi, height, width, length, F0) # compute the value, based on the given parameters
# ##################################


def fitter_func(argument_array, F0, fudge_distance):
    
    height = 1.5
    width = 3   
    length = 21 
    
    return generator_func_arr (F0, fudge_distance, height, width, length, argument_array) #generator_func_single (argument, F0, fudge_distance, height, width, length)
    
    
# ##################################

def least_sqares_method (y_pred, y_label):
    """takes values of two arrays (of equal size) and computes the least squares metric"""
    sum = 0
    
    for i in range(len(y_pred)):
        dif = (y_pred[i] - y_label[i])**2  
        sum = sum + dif
    
    sum = sum/len(y_pred)
    sum = np.sqrt (sum)
    
    return sum     

# ##################################
        
def min_value_parser (threshold, array, sweep_array1 = sweep_arr_length, sweep_array2 = sweep_arr_width):      #
    """This function takes a 2D array and gives one the values and the indeces of where its values are smaller then "threshold" """
    
    bb = np.flatnonzero(array < threshold)
    
    if bb.size == 0:
        print ("===No values below this threshold===")
    
    for i in bb:
        print (i)
        index_2 = i % array.shape[1]
        index_1 = int((i - index_2) / array.shape[1])
        print ("index_1:",index_1,"index_2:",index_2)
        print ('\n',"indexes:", index_1, index_2, "| RMSE value:", array[index_1, index_2])
#        print ("F0 value:", sweep_matrix_F0[index_1], "| Fudge value:", sweep_matrix_fudge[index_2])
        print ("length value:", sweep_array1[index_1], "| Width value:", sweep_array2[index_2])
    
    return None
        
    
###################################

def plotter (x_labels, y_labels, x_predictions, y_predictions):
    
    fig1, (ax1) = plt.subplots(1, 1, figsize = (5, 5), dpi = 400)
    ax1.plot(x_predictions, y_predictions, linewidth=3, label = "fit to detector data", zorder=1)
    
    ax1.scatter(x_labels, y_labels, c='r', linewidths=5, label='Experimental data', zorder=2)
    
    ax1.set_xlabel('$Zenith$ $angle$ ($^{\circ}$)', fontsize=12)
    ax1.set_ylabel('$Absolute$ $count$', fontsize=12)
    # ax1.set_title('$Muon$ $impacts$ $per$ $minute$ $for$ $1x1 cm$ $chip$', fontdict=None, loc='center', pad=None)
    
    ax1.legend()
    
    plt.plot()
    
    return None


##########################################


def namestr(obj, namespace = globals()):
    namespace
    """This function returns the name of an array"""
    return [name for name in namespace if namespace[name] is obj]

###########################################################



# F0 = 0.498
# fudge_distance = 0.1

# # predictions = generator_func (F0, fudge_distance, height, width, length, argument_array)
# # # prediction = generator_func (0, F0, fudge_distance, height, width, length)
# # prediction = fitter_func(5, F0, fudge_distance)


#############################################################

# popt, pcov = curve_fit(fitter_func, argument_array, labels, p0 = [F0, fudge_distance])

 ##########################################################
    

In [ ]:
argument_array = df['angle'].to_numpy()
labels = df['average_cpm_coincidental'].to_numpy()


# Sweep of the height, width and length

In [ ]:
sweep_arr_length = np.linspace (10, 30, 3)
sweep_arr_width = np.linspace (0.5, 3, 3)
sweep_arr_height = np.linspace (1.2, 3.0, 3)
charact_arr2 = np.zeros ((len(sweep_arr_length), len(sweep_arr_width)))
                                                   
F0 = 0.5
fudge_factor = 0.0
                                                   

In [4]:
for i in range(len(sweep_arr_length)):
    print("Progress:", int(100*(i+1)/len(sweep_arr_length)),"%")
    for j in range(len(sweep_arr_width)):
        for k in range(len(sweep_arr_height)):
            
            
            #print (charact_arr[i,j,k], i, j, k)
            y_pred = generator_func_arr (argument_array, F0, fudge_factor, sweep_arr_height[k], sweep_arr_width[j], sweep_arr_length[i])
            charact_arr2[i,j] = least_sqares_method (y_pred, labels)

Progress: 33 %
Progress: 66 %
Progress: 100 %


# Sweeping fudge factor and intensity

In [5]:
sweep_matrix_F0 = np.linspace (0.5,2, 40)
sweep_matrix_fudge = np.linspace (0.2,0.5,20) 
charact_arr = np.zeros ((len(sweep_matrix_F0), len(sweep_matrix_fudge)))

height = 1.5
width = 3   
length = 21

# min_value_parser (2., charact_arr)

In [6]:
for i in range(len(sweep_matrix_F0)):
    print("Progress:", int(100*(i+1)/len(sweep_matrix_F0)),"%")
    for j in range(len(sweep_matrix_fudge)):
        #print (charact_arr[i,j], i, j)
        y_pred = generator_func_arr (argument_array, sweep_matrix_F0[i], sweep_matrix_fudge[j], height, width, length)
        charact_arr[i,j] = least_sqares_method (y_pred, labels)
####################################

Progress: 2 %


C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\quadpack.py:879: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  quad_r = quad(f, low, high, args=args, full_output=self.full_output,


Progress: 5 %
Progress: 7 %
Progress: 10 %
Progress: 12 %


KeyboardInterrupt: 

In [ ]:
grid1_falsecount_correction 
grid1_no_falsecount_correction = charact_arr

In [ ]:
charact_arr

In [41]:
min_value_parser (2.38, charact_arr)  # no correction


 indexes: 29 7 | RMSE value: 2.3536393418421646
F0 value: 0.5974358974358974 | Fudge value: 0.24358974358974358

 indexes: 30 7 | RMSE value: 2.368649805465165
F0 value: 0.6076923076923078 | Fudge value: 0.24358974358974358

 indexes: 32 8 | RMSE value: 2.2961765444010815
F0 value: 0.6282051282051282 | Fudge value: 0.2641025641025641

 indexes: 33 8 | RMSE value: 2.3269753394701365
F0 value: 0.6384615384615384 | Fudge value: 0.2641025641025641

 indexes: 34 9 | RMSE value: 2.329206452940484
F0 value: 0.6487179487179486 | Fudge value: 0.2846153846153846

 indexes: 35 9 | RMSE value: 2.245725789310506
F0 value: 0.658974358974359 | Fudge value: 0.2846153846153846

 indexes: 36 9 | RMSE value: 2.2706102141721725
F0 value: 0.6692307692307693 | Fudge value: 0.2846153846153846

 indexes: 37 10 | RMSE value: 2.302831374115223
F0 value: 0.6794871794871795 | Fudge value: 0.30512820512820515

 indexes: 38 10 | RMSE value: 2.20351216664546
F0 value: 0.6897435897435897 | Fudge value: 0.30512820512